In [7]:
from typing import Callable, List, Type
import sys
sys.path.append('/')
# import gymnasium as gym
import numpy as np
# from mani_skill.envs.sapien_env import BaseEnv
# from mani_skill.utils import common, gym_utils
import argparse
import yaml
sys.path.append(str("/data/ylkong/code/RoboticsDiffusionTransformer-main/scripts"))
from maniskill_model import create_model, RoboticDiffusionTransformerModel
import torch
from collections import deque
from PIL import Image
import cv2
import random
import os

# set cuda 
# args = parse_args()
# set random seeds
seed = 0
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

task2lang={
    "DrawerCleanUp": "Place the cup inside the drawer."
}



ModuleNotFoundError: No module named 'configs'

In [2]:
import argparse

import numpy as np
import robosuite
from robosuite import load_composite_controller_config

# IMPORTANT: you need to import the package to register the environments
import dexmimicgen

ENV_ROBOTS = {
    "TwoArmThreading": ["Panda", "Panda"],
    "TwoArmThreePieceAssembly": ["Panda", "Panda"],
    "TwoArmTransport": ["Panda", "Panda"],
    "TwoArmLiftTray": ["PandaDexRH", "PandaDexLH"],
    "TwoArmBoxCleanup": ["PandaDexRH", "PandaDexLH"],
    "TwoArmDrawerCleanup": ["PandaDexRH", "PandaDexLH"],
    "TwoArmCoffee": ["GR1FixedLowerBody"],
    "TwoArmPouring": ["GR1FixedLowerBody"],
    "TwoArmCanSortRandom": ["GR1ArmsOnly"],
}

[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properly, otherwise you will not be able to use the default IK controller setting for GR1 robot. (__init__.py:40)


In [3]:
env_kwargs = {
    "env_name": "TwoArmDrawerCleanup",
    "robots": ENV_ROBOTS["TwoArmDrawerCleanup"],
    "controller_configs": load_composite_controller_config(
        robot=ENV_ROBOTS["TwoArmDrawerCleanup"][0]
    ),
    "has_renderer": False,
    "has_offscreen_renderer": True,
    "ignore_done": True,
    "use_camera_obs": True,
    "use_object_obs": True,
    "camera_names": ["robot0_eye_in_hand", "robot1_eye_in_hand","agentview"],
    "control_freq": 20,
}

# initialize the task
env = robosuite.make(
    **env_kwargs,
)

[robosuite INFO] Loading controller configuration from: /data/ylkong/robosuite-master/robosuite/controllers/config/robots/default_panda_dex.json (composite_controller_factory.py:121)


In [ ]:
config_path = 'configs/base.yaml'
with open(config_path, "r") as fp:
    config = yaml.safe_load(fp)
pretrained_text_encoder_name_or_path = "google/t5-v1_1-xxl"
pretrained_vision_encoder_name_or_path = "google/siglip-so400m-patch14-384"
pretrained_path = args.pretrained_path
policy = create_model(
    args=config, 
    dtype=torch.bfloat16,
    pretrained=pretrained_path,
    pretrained_text_encoder_name_or_path=pretrained_text_encoder_name_or_path,
    pretrained_vision_encoder_name_or_path=pretrained_vision_encoder_name_or_path
)